# Imports

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
from google.colab import drive
from tensorflow.keras import layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add, Input, MaxPooling2D, GlobalAveragePooling2D, Dense, MaxPool2D
from tensorflow.keras.utils import to_categorical
from keras.layers import ReLU, GlobalAvgPool2D, Dropout
from PIL import Image

drive.mount('/content/drive')

Mounted at /content/drive


Set up the testing images and the training images. Each images pass through a resize and normalization function.

# Lidando com as Imagens

In [ ]:
imagesTesting = []
labelsTesting = []
imagesTraining = []
labelsTraining = []
classLabels = ['barberry', 'bayberry', 'beach plum', 'bearberry', 'black berry',
               'black cherry', 'blueberry', 'ceylon gooseberry', 'chokeberry', 'crowberry']

target_size = (224, 224)

def readTestingImages():
    os.chdir('/content/drive/MyDrive/CNN_SOURCE/Testing/Testing')

    imageCountList = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    for filename in os.listdir():
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Read Image
            image = cv2.imread(filename)

            # Resize the image
            image_resized = cv2.resize(image, target_size)

            # Normalize the image
            image_normalized = image_resized / 255.0

            # Extract label from filename
            labels = filename.rsplit("_", 1)
            label = labels[0].replace('_', ' ')

            # Get the index of the label in classLabels
            label_index = classLabels.index(label)
            imageCountList[label_index] += 1

            if imageCountList[label_index] > 150:
              continue

            imagesTesting.append(image_normalized)
            labelsTesting.append(label_index)

def readTrainingImages():
    os.chdir('/content/drive/MyDrive/CNN_SOURCE/Training/Training')

    imageCountList = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

    for filename in os.listdir():
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Read Image
            image = cv2.imread(filename)

            # Resize the image
            image_resized = cv2.resize(image, target_size)

            # Normalize the image
            image_normalized = image_resized / 255.0

            # Extract label from filename
            labels = filename.rsplit("_", 1)
            label = labels[0].replace('_', ' ')

            # Get the index of the label in classLabels
            label_index = classLabels.index(label)
            imageCountList[label_index] += 1

            if imageCountList[label_index] > 600:
              continue

            imagesTraining.append(image_normalized)
            labelsTraining.append(label_index)

readTestingImages()
readTrainingImages()

Set up the CNN architecture

We start with a architecture of 7 layers. 2 are convolutional layers. 2 are max-pooling layers, a layer to flatten the previous output into a 1D vector, and 2 connected layers (dense layers).

**model = models.Sequential()**: this creates a sequential model, which is a linear stack of layers

**model.add(layers.Vonc2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))**: adds a 2D convolutional layer to the model with 32 filters. The number of filters determine the number of channels of the output feature maps. Each filter produces one feature map, and these feature maps collectively capture different aspects of the input data.

**model.add(layers.MaxPooling2D((2, 2)))**: adds a max-pooling layer to the model with a pool size of 2x2. Max-pooling reduces the spatial dimensions of the feature maps.

**model.add(layers.Flatten())**: this flattens the output of the previous layer into a 1D vector. This reshapes the multi-dimensional feature maps into a 1D vector that can be fed into the fully connected layer. Each element in the vector corresponds to a specific feature or activation in the feature map.

**model.add(layers.Dense(64, activation='relu'))**: This adds a fully connected (dense) layer with 64 units and the ReLU activation function.

**model.add(layers.Dense(10))**: This adds the final fully connected layer with 10 units. Since this is a classification problem with 10 classes, this layer will produce the output logits for each class.

# Arquitetura Resnet

In [ ]:

def create_convulational_layer(x, f, k=1, s=1, p='same'):
    x = Conv2D(f, k, strides=s, padding=p)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return MaxPool2D(3, strides=2, padding='same')(x)

def identity_block(tensor, f):
    x = create_convulational_layer(tensor, f)
    x = create_convulational_layer(x, f, 3)
    x = Conv2D(4*f, 1)(x)
    x = BatchNormalization()(x)

    x = Add()([x, tensor])
    output = ReLU()(x)
    return output

def conv_block(tensor, f, s):
  x = create_convulational_layer(tensor, f)
  x = create_convulational_layer(x, f, 3, s)
  x = Conv2D(4*f, 1)(x)
  x = BatchNormalization()(x)

  shortcut = Conv2D(4*f, 1, strides=s)(tensor)
  shortcut = BatchNormalization()(shortcut)

  x = Add()([x, shortcut])
  output = ReLU()(x)
  return output

def create_resnet_block(x, f, r, s=2):
  x = conv_block(x, f, s)
  for _ in range(r-1):
    x = identity_block(x, f)
  return x

def build_resnet(input_shape, num_classes):
  input = Input(input_shape)

  # Primeira camada convolucional adicional
  x = create_convulational_layer(input, 64, 3, 2)

  # Segunda camada convolucional adicional
  x = create_convulational_layer(x, 64, 3, 1)

  # Terceira camada convolucional adicional
  x = create_convulational_layer(x, 64, 3, 1)

  # Quarta camada convolucional adicional
  x = create_convulational_layer(x, 64, 3, 1)

  # Quinta camada convolucional adicional
  x = create_convulational_layer(x, 64, 3, 1)

  # Sexta camada convolucional adicional
  x = create_convulational_layer(x, 64, 3, 1)

  x = create_resnet_block(x, 64, 3, 1)
  x = create_resnet_block(x, 128, 4)
  x = create_resnet_block(x, 256, 6)
  x = create_resnet_block(x, 512, 3)

  x = GlobalAvgPool2D()(x)

  x = Dropout(0.5)(x)

  output = Dense(num_classes, activation='softmax')(x)

  model = Model(input, output)
  return model

# Construindo Modelo

In [ ]:
# Ajustar os dados de entrada para o formato esperado pela rede
imagesTraining = np.array(imagesTraining).astype('float32')
imagesTesting = np.array(imagesTesting).astype('float32')
imagesTraining /= 255.0
imagesTesting /= 255.0

# Definir o número de classes do problema
num_classes = 10

labelsTraining = np.array(labelsTraining)
labelsTraining = to_categorical(labelsTraining, num_classes)
labelsTesting = np.array(labelsTesting)
labelsTesting = to_categorical(labelsTesting, num_classes)

# Construir o modelo ResNet
input_shape = imagesTraining[0].shape
model = build_resnet(input_shape, num_classes)

# Treinando Modelo

In [ ]:
# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
history = model.fit(imagesTraining, labelsTraining, batch_size=32, epochs=50, validation_data=(imagesTesting, labelsTesting))

Epoch 1/50
63/63 [==============================] - ETA: 0s - loss: 2.7929 - accuracy: 0.1975 

# Avaliação do Modelo

In [ ]:
# Avaliação do modelo
test_loss, test_acc = model.evaluate(imagesTesting, labelsTesting)
print('Acurácia final do modelo: %.2f%%' % (test_acc * 100))

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.05, 1])
plt.legend(loc='lower right')

In [ ]:
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.ylim([0.1, 30])
plt.legend(loc='lower right')

# Salvando Modelo

In [ ]:
# Salvar o modelo
model.save('/content/drive/MyDrive/CNN_SOURCE/Modelos/cnn_modelo_resnet.h5')